In [1]:
! pip install pytorch-pretrained-bert

In [3]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
 
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)
 
import matplotlib.pyplot as plt

 
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
def GetBertEmbeddings(text):
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1] * len(tokenized_text)
  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  batch_i=0
# Load pre-trained model (weights)
  model = BertModel.from_pretrained('bert-base-uncased')
 
# Put the model in "evaluation" mode, meaning feed-forward operation.
  model.eval()
  # Predict hidden states features for each layer
  with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
  # Convert the hidden state embeddings into single token vectors
 
# Holds the list of 12 layer embeddings for each token
# Will have the shape: [# tokens, # layers, # features]
  token_embeddings = [] 
 
# For each token in the sentence...
  for token_i in range(len(tokenized_text)):
  
  # Holds 12 layers of hidden states for each token 
    hidden_layers = [] 
  
  # For each of the 12 layers...
    for layer_i in range(len(encoded_layers)):
    
    # Lookup the vector for `token_i` in `layer_i`
      vec = encoded_layers[layer_i][batch_i][token_i]
    
      hidden_layers.append(vec)
    
      token_embeddings.append(hidden_layers)
    
  # Predict hidden states features for each layer
  summed_last_4_layers = [torch.sum(torch.stack(layer)[-4:], 0) for layer in token_embeddings] # [number_of_tokens, 768]
  return summed_last_4_layers

 

  


In [5]:
GetBertEmbeddings("trial")

    -2.5672e-01, -9.8750e-01,  6.7846e-02, -2.3129e-01, -8.5311e-01,
          3.7924e-01,  8.7818e-02, -9.0597e-01,  8.7486e-01,  2.0236e-01,
         -1.7406e-01, -9.2567e-02,  2.8902e-01, -3.8748e-01,  1.0767e-01,
         -2.5177e-02, -3.2177e-01, -4.0216e-01,  3.5885e-01, -7.0211e-02,
         -1.4778e+00,  7.8578e-02, -4.0299e-01,  2.7286e-01,  3.5921e-01,
         -9.6176e-01,  1.2518e-01,  1.7300e-01, -8.5785e-01,  3.3212e-01,
         -3.4489e-01,  4.3356e-01, -3.9391e-01, -1.1202e+00,  1.0622e-01,
         -4.7606e-01, -8.7947e-01,  7.8354e-02, -2.6937e-01,  3.2232e-01,
         -3.1477e+00,  3.1462e-01,  1.1419e+00,  6.2978e-01,  2.3107e-01,
          1.8657e-01,  3.0649e-02,  6.0263e-01,  1.7660e-01,  3.9254e-02,
          1.4880e-01,  3.7656e-01, -1.5543e-01, -5.7990e-01, -2.9876e-01,
         -4.0783e-01,  2.6563e-01,  1.7412e-01, -6.5444e-01, -2.3708e-01,
         -8.1073e-01, -2.7964e-02,  8.3467e-01,  1.0736e+00, -4.3779e-01,
          1.4137e-01, -8.1362e-01,  3.8825e